# SmartSwitch Tutorial 02: Hierarchical Names

**Welcome back!** This is the second tutorial in the SmartSwitch series.

In this notebook you'll learn:
- ✅ How to organize handlers with prefixes
- ✅ How to use dot notation for hierarchies
- ✅ How to build scalable API structures

**Time**: ~7 minutes

**Prerequisites**: Complete Tutorial 01 first

---

## The Problem

You have many handlers and need a way to organize them.

**Without prefixes** - Name collisions:

In [ ]:
from smartswitch import Switcher

api = Switcher()

@api
def user_create(data):
    return f"Creating user: {data}"

@api
def user_delete(user_id):
    return f"Deleting user: {user_id}"

@api
def product_create(data):
    return f"Creating product: {data}"

@api
def product_delete(product_id):
    return f"Deleting product: {product_id}"

# Function names are verbose!
print(api('user_create')({'name': 'Alice'}))
print(api.entries())

**Problems:**
- ❌ Repetitive prefixes in function names (`user_`, `product_`)
- ❌ No logical grouping
- ❌ Difficult to discover what operations exist for each resource

## Solution 1: Prefix-Based Auto-Naming

Let SmartSwitch strip the prefix automatically:

In [ ]:
from smartswitch import Switcher

# Set a prefix - it will be stripped from handler names!
users = Switcher(prefix="user_")

@users
def user_create(data):
    return f"Creating user: {data}"

@users
def user_delete(user_id):
    return f"Deleting user: {user_id}"

@users
def user_list():
    return ["Alice", "Bob"]

# Call with SHORT names (prefix stripped!)
print(users('create')({'name': 'Charlie'}))
print(users('list')())

# See the registered names
print("\nRegistered:", users.entries())

**Benefits:**
- ✅ Keep function names descriptive (`user_create`)
- ✅ Call with short names (`'create'`)
- ✅ Convention-driven naming

## Solution 2: Hierarchical Switchers

Organize multiple Switchers in a class:

In [ ]:
from smartswitch import Switcher

class MyAPI:
    # Main switcher
    mainswitch = Switcher(name="main")
    
    # Child switchers with prefixes
    users = mainswitch.add(Switcher(name="users", prefix="user_"))
    products = mainswitch.add(Switcher(name="products", prefix="product_"))
    
    @users
    def user_create(self, data):
        return f"Creating user: {data}"
    
    @users
    def user_list(self):
        return ["Alice", "Bob"]
    
    @products
    def product_create(self, data):
        return f"Creating product: {data}"
    
    @products
    def product_list(self):
        return ["Laptop", "Phone"]

# Use it
api = MyAPI()

# Direct access via child switchers
print(api.users('create')({'name': 'Charlie'}))
print(api.products('list')())

## Dot Notation Access

Access handlers through the hierarchy with dot notation:

In [ ]:
# Same API as above

# Via mainswitch with dot notation!
print(api.mainswitch('users.create')({'name': 'Dave'}))
print(api.mainswitch('products.list')())

# This is especially useful for:
# - Generic handlers that dispatch based on path
# - CLI tools with nested commands
# - API gateways

## Discovery: What's Available?

Discover all child switchers and their handlers:

In [ ]:
# List all child switchers
print("Child switchers:")
for child in api.mainswitch.children:
    print(f"\n{child.name}:")
    for handler in child.entries():
        print(f"  - {handler}")

## Try It Yourself!

Build a blog API with posts and comments:

In [ ]:
from smartswitch import Switcher

class BlogAPI:
    mainswitch = Switcher(name="blog")
    
    posts = mainswitch.add(Switcher(name="posts", prefix="post_"))
    comments = mainswitch.add(Switcher(name="comments", prefix="comment_"))
    
    @posts
    def post_create(self, title, content):
        return f"Post created: {title}"
    
    # TODO: Add more handlers!
    # @posts
    # def post_list(self): ...
    
    # @comments
    # def comment_create(self, post_id, text): ...

blog = BlogAPI()
print(blog.posts('create')('My First Post', 'Hello World!'))

## Real-World Example: Multi-Resource API

A complete REST-like API structure:

In [ ]:
from smartswitch import Switcher

class RestAPI:
    root = Switcher(name="api")
    
    users = root.add(Switcher(name="users", prefix="user_"))
    posts = root.add(Switcher(name="posts", prefix="post_"))
    auth = root.add(Switcher(name="auth", prefix="auth_"))
    
    @users
    def user_create(self, data):
        return {"id": 1, "name": data["name"]}
    
    @users
    def user_get(self, user_id):
        return {"id": user_id, "name": "Alice"}
    
    @posts
    def post_list(self):
        return [{"id": 1, "title": "First Post"}]
    
    @auth
    def auth_login(self, username, password):
        return {"token": "abc123"}

api = RestAPI()

# Simulate API calls
print("Create user:", api.users('create')({'name': 'Bob'}))
print("Get user:", api.users('get')(1))
print("List posts:", api.posts('list')())
print("Login:", api.auth('login')('alice', 'secret'))

# Or via root with dot notation
print("\nVia dot notation:")
print(api.root('users.get')(2))

## Pattern: Automatic API Registration

Use parent-child relationships for framework-style registration:

In [ ]:
# Root API registry
root_api = Switcher(name="root")

# Handler classes auto-register themselves
class UserHandler:
    api = Switcher(name="users", prefix="user_", parent=root_api)
    
    @api
    def user_list(self):
        return ["alice", "bob"]
    
    @api
    def user_get(self, name):
        return f"User: {name}"

class PostHandler:
    api = Switcher(name="posts", prefix="post_", parent=root_api)
    
    @api
    def post_list(self):
        return ["post1", "post2"]

# Discovery: iterate all registered modules
print("Registered API modules:")
for child in root_api.children:
    print(f"\n{child.name}:")
    for handler in child.entries():
        print(f"  - {handler}")

## When to Use Hierarchical Names

This pattern is perfect for:

✅ **Multi-resource APIs** - REST APIs, GraphQL resolvers

✅ **Plugin architectures** - Auto-register modules

✅ **Large applications** - Organize hundreds of handlers

✅ **Framework building** - Provide extensibility points

⚠️ **Overkill for**:
- Small APIs with <10 handlers
- When all handlers belong to one domain

## Exercise: Build a CLI Tool

Create a git-like CLI with nested commands:
- `config set <key> <value>`
- `config get <key>`
- `remote add <name> <url>`
- `remote list`

In [ ]:
from smartswitch import Switcher

class GitCLI:
    root = Switcher(name="git")
    
    config = root.add(Switcher(name="config", prefix="config_"))
    remote = root.add(Switcher(name="remote", prefix="remote_"))
    
    # TODO: Implement commands!
    @config
    def config_set(self, key, value):
        return f"Set {key} = {value}"
    
    # Add more...

git = GitCLI()

# Test
print(git.config('set')('user.name', 'Alice'))
# print(git.root('remote.add')('origin', 'https://...'))

## Summary

You learned:

✅ **Prefix auto-naming** - Strip prefixes with `prefix="user_"`

✅ **Hierarchical organization** - Group with `mainswitch.add()`

✅ **Dot notation** - Access via `mainswitch('users.create')`

✅ **Discovery** - Find handlers with `children` and `entries()`

---

## Next Steps

Continue to **Tutorial 03: Value-Based Dispatch** to learn how to automatically route based on runtime values instead of calling handlers by name.

📖 **Documentation**: [API Discovery Guide](https://smartswitch.readthedocs.io/guide/api-discovery/)

---

**Questions?** Open an issue on [GitHub](https://github.com/genropy/smartswitch/issues)